In [1]:
# import libraries
import ee, ssl
# bypass certificate error: this might only be an issue on my computer
# ssl._create_default_https_context = ssl._create_unverified_context
# Authenticate and initialize the library.
# ee.Authenticate() # only need to run this once 
ee.Initialize() 

In [2]:
# Pull GEE pollution data for specified years!

region = ee.Geometry.Polygon(coords=
    [[[-74.89788681934836, 46.17955456708541],
    [-74.89788681934836, 45.03443324885425],
    [-72.31609970997336, 45.03443324885425],
    [-72.31609970997336, 46.17955456708541]]], proj=None) # The rough extent of our study region 

# Create list of the pollutants that we wish to study

poll = [
  ['COPERNICUS/S5P/NRTI/L3_NO2', "NO2_column_number_density", 'L3_NO2'], # Nitrogen Dioxide
  ['COPERNICUS/S5P/NRTI/L3_CO', "CO_column_number_density", 'L3_CO'], # Carbon Monoxide
  ['COPERNICUS/S5P/NRTI/L3_HCHO', "tropospheric_HCHO_column_number_density", "L3_HCHO"], # Formaldehyde
  ['COPERNICUS/S5P/NRTI/L3_SO2', "SO2_column_number_density", "L3_SO"] # Sulfur dioxide
]

# make list of study years 

year = [2018, 2019, 2020, 2021]

# iterate over the desired pollution data
for j in year:
  for i in poll: #for loop that iterates over the pollution list
      collection = ee.ImageCollection(i[0]) \
      .filterDate(str(j) + '-09-01', str(j) + '-12-31') \
      .filterBounds(region) \
      .select(i[1]) # select the pollution band 
      meanImage = collection.mean() # calculate the mean of the images
      scale = collection.first().select(i[1]).projection().nominalScale() # get the scale of each image
      # Export the image to your Google Drive: look at the task tab in Google Earth Engine
      ee.batch.Export.image.toDrive(image=meanImage,
                                    folder='gee_pollution',    
                                    description=i[2]+'_'+str(j),
                                    scale=scale,
                                    fileNamePrefix=i[2]+'_'+str(j),
                                    fileFormat='GeoTIFF', 
                                    region=region['coordinates'], 
                                    ).start()
      